In [2]:
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing as pre
import scipy.io as sio
from scipy import signal
import math
from scipy.signal import detrend
from scipy.stats import entropy, kurtosis, skew
from sklearn.feature_selection import f_classif, VarianceThreshold
import collections
import scipy.stats as stats
import cupy as cp
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
import scipy.interpolate as interp
from sklearn.impute import SimpleImputer
import pywt

In [4]:
raw = sio.loadmat('./DREAMER_Dataset/DREAMER.mat')

In [5]:
# create new dataframe with emotion, participant, and video data
a=np.zeros((23,18,9),dtype=object)
for participant in range(0,23):
    for video in range(0,18):
        a[participant,video,0]=raw['DREAMER'][0,0]['Data'][0,participant]['Age'][0][0][0]
        a[participant,video,1]=raw['DREAMER'][0,0]['Data'][0,participant]['Gender'][0][0][0]
        a[participant,video,2]=participant+1
        a[participant,video,3]=video+1
        a[participant,video,4]=['Searching for Bobby Fischer','D.O.A.', 'The Hangover', 'The Ring', '300',
                  'National Lampoon\'s VanWilder', 'Wall-E', 'Crash', 'My Girl', 'The Fly',
                  'Pride and Prejudice', 'Modern Times', 'Remember the Titans', 'Gentlemans Agreement',
                  'Psycho', 'The Bourne Identitiy', 'The Shawshank Redemption', 'The Departed'][video]
        a[participant,video,5]=['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                  'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                  'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]
        a[participant,video,6]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreValence'][0,0][video,0]
        a[participant,video,7]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreArousal'][0,0][video,0]
        a[participant,video,8]=raw['DREAMER'][0,0]['Data'][0,participant]['ScoreDominance'][0,0][video,0]
b=pd.DataFrame(a.reshape((23*18,a.shape[2])),columns=['Age','Gender','Participant','Video','Video_Name','Target_Emotion','Valence','Arousal','Dominance'])

In [6]:
b

,Age,Gender,Participant,Video,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,22,male,1,2,D.O.A.,surprise,3,3,1
2,22,male,1,3,The Hangover,amusement,5,4,4
3,22,male,1,4,The Ring,fear,4,3,2
4,22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...
409,25,male,23,14,Gentlemans Agreement,anger,2,2,2
410,25,male,23,15,Psycho,fear,2,2,2
411,25,male,23,16,The Bourne Identitiy,excitement,3,3,2
412,25,male,23,17,The Shawshank Redemption,sadness,2,2,4


## DWT decomposition

In [6]:
# Function to perform DWT on EEG data and extract frequency bands
def dwt_and_band_extraction(inputAr, eeg_data):
    # Perform DWT with db5 wavelet, 4 levels
    coeffs = pywt.wavedec(cp.asnumpy(eeg_data), 'db5', level=4)
    
    # Extract approximation (cA) and detail coefficients (cD)
    cA4, cD4, cD3, cD2, cD1 = coeffs
    
    # Reconstruct the signals for each band
    theta = cD4
    alpha = cD3
    beta = cD2
    gamma = cD1
    
    inputAr.append(theta)
    inputAr.append(alpha)
    inputAr.append(beta)
    inputAr.append(gamma)
    return inputAr

## Preprocessing

In [7]:
def detrend_signal(signal):
    return cp.array(detrend(cp.asnumpy(signal), axis=-1))

def baseline_correction(baseline, stimuli):
    baseline_mean = cp.mean(baseline, axis=-1, keepdims=True)
    return stimuli - baseline_mean

## EMD decomposition

In [7]:
import numpy as np
from PyEMD import CEEMDAN
from scipy.signal import resample
import matplotlib.pyplot as plt

def process_eeg_signals(eeg_signals, fixed_length):
    ceemdan = CEEMDAN()
    processed_imfs = []

    for eeg_signal in eeg_signals:
        imfs = ceemdan(eeg_signal)
        resampled_imfs = []

        for imf in imfs:
            if len(imf) > fixed_length:
                # Truncate the IMF
                imf = imf[:fixed_length]
            elif len(imf) < fixed_length:
                # Resample the IMF to the fixed length
                imf = resample(imf, fixed_length)
            resampled_imfs.append(imf)

        processed_imfs.append(resampled_imfs)
    
    return processed_imfs

# Example EEG signals (replace with actual EEG data)
eeg_signals = [np.sin(2 * np.pi * 5 * np.linspace(0, 1, 200)) + np.random.normal(0, 0.5, 200) for _ in range(10)]
fixed_length = 100  # Desired fixed length for each IMF

# Process EEG signals
processed_imfs = process_eeg_signals(eeg_signals, fixed_length)

# Plot the first IMF of the first signal to check the result
plt.figure(figsize=(10, 6))
plt.plot(processed_imfs[0][0])
plt.title('First IMF of the First EEG Signal (Resampled/Truncated)')
plt.show()

ModuleNotFoundError: No module named 'PyEMD'

In [8]:
def find_extrema(signal):
    maxima = (cp.diff(cp.sign(cp.diff(signal))) < 0).nonzero()[0] + 1  # local max
    minima = (cp.diff(cp.sign(cp.diff(signal))) > 0).nonzero()[0] + 1  # local min
    return maxima, minima


def interpolate_extrema(x, signal, extrema):
    t = np.arange(len(signal))
    interpolator = interp.CubicSpline(cp.asnumpy(extrema), cp.asnumpy(signal)[cp.asnumpy(extrema)])
    return cp.array(interpolator(t))


def emd(signal, max_imfs=6):
    IMFs = []
    signal = cp.asnumpy(signal)
    residue = signal
    for _ in range(max_imfs):
        h = residue
        for _ in range(1000):  # Sifting process, with a maximum of 1000 iterations
            maxima, minima = find_extrema(h)
            if len(maxima) < 2 or len(minima) < 2:  # Break if too few extrema
                break
            upper_env = interpolate_extrema(np.arange(len(h)), h, maxima)
            lower_env = interpolate_extrema(np.arange(len(h)), h, minima)
            mean_env = (upper_env + lower_env) / 2
            prev_h = h
            h = cp.array(h) - cp.array(mean_env)

            # Check if h is an IMF
            if cp.allclose(prev_h, h, atol=1e-10):
                break

        IMFs.append(cp.asnumpy(h))
        residue = cp.array(residue) - h
        if len(find_extrema(residue)[0]) < 2:
            break
    
    # Ensure exactly 6 IMFs by padding with zeros if necessary
    while len(IMFs) < max_imfs:
        IMFs.append(np.zeros_like(signal))

    return IMFs 

In [9]:
eeg_electrodes = [str(e[0]) for e in raw['DREAMER']['EEG_Electrodes'][0][0][0]]

### Trial feature extraction just for EMD of EEG signals

In [ ]:
def feature_extraction_EEG(raw,secs):
    # 128 Hz is the sampling rate for the EEG data
    fs_EEG = 128
    N_EEG = math.ceil(fs_EEG*secs)
    features = []
    for electrode in eeg_electrodes:
        features += [f'{electrode}_IMF_{i+1}' for i in range(6)]
    # DataFrame to store features
    columns = ['participant', 'video'] + features
    df = pd.DataFrame(columns=columns)
    for participant in range(23):
        for video in range(18):
            feature_values = []
            for i in range(14):
                basl=raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['baseline'][0,0][video,0][-1-N_EEG:-1,i]
                stimuli = raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['stimuli'][0,0][video,0][-1-N_EEG:-1,i]
                # Detrending
                # basl_detrended = detrend_signal(basl)
                # stimuli_detrended = detrend_signal(stimuli)
                
                # Baseline correction
                # stimuli_corrected = baseline_correction(basl_detrended, stimuli_detrended)
                
                # Feature extraction
                # DWT_stimuli = dwt_and_band_extraction([], stimuli_corrected)
                emd_stimuli = emd(stimuli)

                # Combine features
                feature_values += emd_stimuli
            
            # Append to DataFrame
            row = [participant+1, video+1] + feature_values
            if len(row) == len(columns):
                df.loc[len(df)] = row
            else:
                print(f'Skipping participant {participant+1}, video {video+1}')
                print(len(row))
                print(row)
                print(len(columns))
                print(columns)
    return df

In [10]:
def feature_extraction_EEG(raw,secs):
    # 128 Hz is the sampling rate for the EEG data
    fs_EEG = 128
    N_EEG = math.ceil(fs_EEG*secs)
    features = []
    for electrode in eeg_electrodes:
        features += [f'{electrode}_theta', f'{electrode}_alpha', f'{electrode}_beta', f'{electrode}_gamma'] + [f'{electrode}_IMF_{i+1}' for i in range(6)]
    # DataFrame to store features
    columns = ['participant', 'video'] + features
    df = pd.DataFrame(columns=columns)
    for participant in range(23):
        for video in range(18):
            feature_values = []
            for i in range(14):
                basl=raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['baseline'][0,0][video,0][-1-N_EEG:-1,i]
                stimuli = raw['DREAMER'][0,0]['Data'][0,participant]['EEG'][0,0]['stimuli'][0,0][video,0][-1-N_EEG:-1,i]
                # Detrending
                basl_detrended = detrend_signal(basl)
                stimuli_detrended = detrend_signal(stimuli)
                
                # Baseline correction
                stimuli_corrected = baseline_correction(basl_detrended, stimuli_detrended)
                
                # Feature extraction
                DWT_stimuli = dwt_and_band_extraction([], stimuli_corrected)
                emd_stimuli = emd(stimuli_corrected)

                # Combine features
                feature_values += DWT_stimuli + emd_stimuli
            
            # Append to DataFrame
            row = [participant+1, video+1] + feature_values
            if len(row) == len(columns):
                df.loc[len(df)] = row
            else:
                print(f'Skipping participant {participant+1}, video {video+1}')
                print(len(row))
                print(row)
                print(len(columns))
                print(columns)
    return df

In [ ]:
# Extract features
secs = 60  # Number of seconds for the EEG data segment
initial_df = feature_extraction_EEG(raw, secs)
initial_df.head()

In [12]:
initial_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_theta,AF4_alpha,AF4_beta,AF4_gamma,AF4_IMF_1,AF4_IMF_2,AF4_IMF_3,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6
0,1,1,"[-0.0390636464244261, 0.09988633859337229, -0....","[0.15276303577969383, 1.2696051548240934, 1.04...","[1.856929710807957, 10.791781431213384, 14.034...","[3.2594967503673904, -1.796087448369993, -0.32...","[-20.945664729800182, -3.585145972089575, 4.35...","[20.079570340536844, 12.845106670083682, 7.499...","[-472.3937889410268, -441.7314862942789, -412....","[807.2982350705635, 760.702080493103, 715.9087...",...,"[7.312922223145134, 90.23369332473663, 49.0698...","[9.73992741999544, -42.45120101008684, 61.3384...","[-3.68186027314305, -72.1481987191482, -94.482...","[-21.046898665876007, 10.740292041546054, 7.63...","[80.0119050099506, -9.449980826652972, -46.883...","[1037.206448963144, 950.0841781770687, 856.204...","[-2836.5958547846376, -2634.3254987093683, -24...","[4147.6440511798155, 3958.769781567537, 3775.2...","[30008.783296321082, 29555.48005523474, 29105....","[-31894.473104879136, -31357.49786213323, -308..."
1,1,2,"[0.4447054345237156, -2.2739034949040344, 2.20...","[-1.29806126965089, -10.057003013278859, -7.98...","[0.049750625457719196, 16.734604267689296, 5.2...","[2.059528878325656, -2.1951011266790372, 2.666...","[3.2907522198672656, 9.558369126486213, 5.4292...","[-68.50388036946528, -60.83343124236948, -53.8...","[287.55871187843275, 265.7149737780089, 244.93...","[-306.1977954610037, -285.01993005917274, -264...",...,"[-9.435727288188353, -104.2044744416575, -58.5...","[-8.024643029620332, 78.39285347896043, -48.87...","[-7.052696424814903, -37.78562877122805, 8.701...","[-2.356633611215584, 4.462868695288647, -9.906...","[16.43269402476068, -17.59069955184509, -14.27...","[-540.2093850563257, -472.7924553125366, -413....","[-2398.7035133969575, -2302.175449310355, -220...","[6703.707191070769, 6417.0011542154925, 6138.8...","[-5458.341931405286, -5260.693608408112, -5067...","[3870.6201793020477, 3777.912123469, 3686.6973..."
2,1,3,"[0.1450676936889263, 0.3323767388122518, 0.725...","[-0.10225776435297178, -1.8531720250660215, -0...","[-1.5964275091820885, -9.08784206257223, -14.0...","[-3.2885028522031257, 0.39521514219349035, 3.5...","[4.572573342644084, -2.958668674311242, -8.078...","[-58.45535598399263, -49.54152737215601, -41.5...","[179.6804531547695, 158.94030718637714, 139.92...","[-158.39189250236828, -145.03333315846984, -13...",...,"[-0.72839340668449, -14.230240869951386, -7.48...","[-2.2453442008337383, -0.7608959816109886, -14...","[-0.8177342172566555, 12.082695315975077, 6.93...","[1.8245642291362119, -2.404378702967243, -3.83...","[1109.288612480044, 920.6166170590583, 742.368...","[1685.5264607560403, 1536.665984683067, 1397.8...","[-6075.212896231042, -5548.437262916795, -5056...","[3140.933258696317, 2967.9150749194423, 2801.2...","[-28134.659465270382, -27597.35851561921, -270...","[40224.09867824291, 39483.0549741928, 38751.11..."
3,1,4,"[0.02241420804875787, 1.1307988471909174, 0.12...","[0.12598080801138814, -0.20322530118359053, 0....","[2.091279813251027, 13.599228413503267, 15.936...","[3.9362192425632094, -1.4331300435338477, -0.6...","[-14.13033785077942, 1.6262765249300335, 8.636...","[317.9041468923732, 278.43980546262344, 242.82...","[-171.79459950088759, -148.3269975635026, -126...","[-171.59902290367822, -156.8806113223296, -143...",...,"[3.6550195243212875, 68.83365942147277, 24.179...","[9.448429372215577, -7.7273928361771285, 56.99...","[-0.49114672423128725, -70.62508671942511, -60...","[-14.580320992236327, 12.148266224535629, 1.51...","[18.091785817921526, -26.826152706666555, -27....","[723.9702983568445, 610.5925850812206, 508.207...","[4345.881956541403, 4132.499268343777, 3922.30...","[-5145.955206893975, -4828.527064582459, -4524...","[-76466.47993581026, -75346.22839802949, -7423...","[76236.82859903804, 75122.73916055362, 74019.5..."
4,1,5,"[1.07091

In [20]:
len(initial_df["AF4_IMF_2"].iloc[0])

7680

In [21]:
# Merge the feature DataFrame with metadata DataFrame
merged_df = pd.merge(initial_df, b, left_on=['participant', 'video'], right_on=['Participant', 'Video'])

In [22]:
merged_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_IMF_6,Age,Gender,Participant,Video,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,1,1,"[-0.0390636464244261, 0.09988633859337229, -0....","[0.15276303577969383, 1.2696051548240934, 1.04...","[1.856929710807957, 10.791781431213384, 14.034...","[3.2594967503673904, -1.796087448369993, -0.32...","[-20.945664729800182, -3.585145972089575, 4.35...","[20.079570340536844, 12.845106670083682, 7.499...","[-472.3937889410268, -441.7314862942789, -412....","[807.2982350705635, 760.702080493103, 715.9087...",...,"[-31894.473104879136, -31357.49786213323, -308...",22,male,1,1,Searching for Bobby Fischer,calmness,4,3,2
1,1,2,"[0.4447054345237156, -2.2739034949040344, 2.20...","[-1.29806126965089, -10.057003013278859, -7.98...","[0.049750625457719196, 16.734604267689296, 5.2...","[2.059528878325656, -2.1951011266790372, 2.666...","[3.2907522198672656, 9.558369126486213, 5.4292...","[-68.50388036946528, -60.83343124236948, -53.8...","[287.55871187843275, 265.7149737780089, 244.93...","[-306.1977954610037, -285.01993005917274, -264...",...,"[3870.6201793020477, 3777.912123469, 3686.6973...",22,male,1,2,D.O.A.,surprise,3,3,1
2,1,3,"[0.1450676936889263, 0.3323767388122518, 0.725...","[-0.10225776435297178, -1.8531720250660215, -0...","[-1.5964275091820885, -9.08784206257223, -14.0...","[-3.2885028522031257, 0.39521514219349035, 3.5...","[4.572573342644084, -2.958668674311242, -8.078...","[-58.45535598399263, -49.54152737215601, -41.5...","[179.6804531547695, 158.94030718637714, 139.92...","[-158.39189250236828, -145.03333315846984, -13...",...,"[40224.09867824291, 39483.0549741928, 38751.11...",22,male,1,3,The Hangover,amusement,5,4,4
3,1,4,"[0.02241420804875787, 1.1307988471909174, 0.12...","[0.12598080801138814, -0.20322530118359053, 0....","[2.091279813251027, 13.599228413503267, 15.936...","[3.9362192425632094, -1.4331300435338477, -0.6...","[-14.13033785077942, 1.6262765249300335, 8.636...","[317.9041468923732, 278.43980546262344, 242.82...","[-171.79459950088759, -148.3269975635026, -126...","[-171.59902290367822, -156.8806113223296, -143...",...,"[76236.82859903804, 75122.73916055362, 74019.5...",22,male,1,4,The Ring,fear,4,3,2
4,1,5,"[1.0709170021849068, 4.750019879969691, 6.0586...","[-0.6217449925448907, -14.325770276644187, -3....","[1.069408056436174, 21.515804657065107, 8.8033...","[3.143577123732196, -2.62735610202805, 2.87790...","[5.444857516311669, 13.90708797820107, 9.48599...","[80.0422016774792, 69.74254636781487, 59.82124...","[-116.43984695695993, -99.4370175539302, -84.0...","[666.7787873551133, 635.2689333389025, 604.836...",...,"[-3763.168998998994, -3683.7473519467703, -360...",22,male,1,5,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[-1.6257516966485197, -22.549341769905883, -10...","[-1.8307911457716481, 14.076797310493374, -11....","[-1.7194771272843405, -6.5520805882942765, 0.0...","[-0.7243271988767334, 1.0256966662912683, 0.33...","[-89.82175150696582, -65.30009587833204, -41.5...","[258.17014084945714, 204.57960244020467, 159.7...","[-179.44886447907598, -151.25541164395247, -12...","[-148.7429926265613, -143.49484989029142, -138...",...,"[-1980.9911196019332, -1912.0766249830497, -18...",25,male,23,14,Gentlemans Agreement,anger,2,2,2
410,23,15,"[-0.34769011135992756, -9.896798418848553, -2....","[-1.976691111166075, -4.525220086587955, -12.1...","[-2.1216863281309988, 3.314999480896199, -6.44...","[-1.124054294663887, -0.6437197702755763, 0.64...","[-167.7639086196857, -121.12071769633161, -86....","[41.78254603681751, 12.82407463089767, -7.7221...","[-954.6212085083736, -887.3263545230769, -822....","[1112.0121639968852, 1023.5093391771265, 939.7...",...,"[-718.4133850112005, -699.080046100827, -680.0...",25,male,23,15,Psycho,fear,2,2,2
411,23,16,"[0.7362872104360669, 15.605776742678449, 4.904...","[2.2479798900065724, -0.4695

In [23]:
final_df = merged_df.drop(["Participant", "Video"], axis=1)
final_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_IMF_4,AF4_IMF_5,AF4_IMF_6,Age,Gender,Video_Name,Target_Emotion,Valence,Arousal,Dominance
0,1,1,"[-0.0390636464244261, 0.09988633859337229, -0....","[0.15276303577969383, 1.2696051548240934, 1.04...","[1.856929710807957, 10.791781431213384, 14.034...","[3.2594967503673904, -1.796087448369993, -0.32...","[-20.945664729800182, -3.585145972089575, 4.35...","[20.079570340536844, 12.845106670083682, 7.499...","[-472.3937889410268, -441.7314862942789, -412....","[807.2982350705635, 760.702080493103, 715.9087...",...,"[4147.6440511798155, 3958.769781567537, 3775.2...","[30008.783296321082, 29555.48005523474, 29105....","[-31894.473104879136, -31357.49786213323, -308...",22,male,Searching for Bobby Fischer,calmness,4,3,2
1,1,2,"[0.4447054345237156, -2.2739034949040344, 2.20...","[-1.29806126965089, -10.057003013278859, -7.98...","[0.049750625457719196, 16.734604267689296, 5.2...","[2.059528878325656, -2.1951011266790372, 2.666...","[3.2907522198672656, 9.558369126486213, 5.4292...","[-68.50388036946528, -60.83343124236948, -53.8...","[287.55871187843275, 265.7149737780089, 244.93...","[-306.1977954610037, -285.01993005917274, -264...",...,"[6703.707191070769, 6417.0011542154925, 6138.8...","[-5458.341931405286, -5260.693608408112, -5067...","[3870.6201793020477, 3777.912123469, 3686.6973...",22,male,D.O.A.,surprise,3,3,1
2,1,3,"[0.1450676936889263, 0.3323767388122518, 0.725...","[-0.10225776435297178, -1.8531720250660215, -0...","[-1.5964275091820885, -9.08784206257223, -14.0...","[-3.2885028522031257, 0.39521514219349035, 3.5...","[4.572573342644084, -2.958668674311242, -8.078...","[-58.45535598399263, -49.54152737215601, -41.5...","[179.6804531547695, 158.94030718637714, 139.92...","[-158.39189250236828, -145.03333315846984, -13...",...,"[3140.933258696317, 2967.9150749194423, 2801.2...","[-28134.659465270382, -27597.35851561921, -270...","[40224.09867824291, 39483.0549741928, 38751.11...",22,male,The Hangover,amusement,5,4,4
3,1,4,"[0.02241420804875787, 1.1307988471909174, 0.12...","[0.12598080801138814, -0.20322530118359053, 0....","[2.091279813251027, 13.599228413503267, 15.936...","[3.9362192425632094, -1.4331300435338477, -0.6...","[-14.13033785077942, 1.6262765249300335, 8.636...","[317.9041468923732, 278.43980546262344, 242.82...","[-171.79459950088759, -148.3269975635026, -126...","[-171.59902290367822, -156.8806113223296, -143...",...,"[-5145.955206893975, -4828.527064582459, -4524...","[-76466.47993581026, -75346.22839802949, -7423...","[76236.82859903804, 75122.73916055362, 74019.5...",22,male,The Ring,fear,4,3,2
4,1,5,"[1.0709170021849068, 4.750019879969691, 6.0586...","[-0.6217449925448907, -14.325770276644187, -3....","[1.069408056436174, 21.515804657065107, 8.8033...","[3.143577123732196, -2.62735610202805, 2.87790...","[5.444857516311669, 13.90708797820107, 9.48599...","[80.0422016774792, 69.74254636781487, 59.82124...","[-116.43984695695993, -99.4370175539302, -84.0...","[666.7787873551133, 635.2689333389025, 604.836...",...,"[-19655.857959376175, -19011.230077163775, -18...","[19107.90560091094, 18491.577630805412, 17888....","[-3763.168998998994, -3683.7473519467703, -360...",22,male,300,excitement,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[-1.6257516966485197, -22.549341769905883, -10...","[-1.8307911457716481, 14.076797310493374, -11....","[-1.7194771272843405, -6.5520805882942765, 0.0...","[-0.7243271988767334, 1.0256966662912683, 0.33...","[-89.82175150696582, -65.30009587833204, -41.5...","[258.17014084945714, 204.57960244020467, 159.7...","[-179.44886447907598, -151.25541164395247, -12...","[-148.7429926265613, -143.49484989029142, -138...",...,"[6287.741777107379, 5985.9936816854615, 5693.5...","[-5691.052134649419, -5370.406783126041, -5061...","[-1980.9911196019332, -1912.0766249830497, -18...",25,male,Gentlemans Agreement,anger,2,2,2
410,2

In [24]:
import pickle
with open('Extracted_Features/data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(final_df, f, pickle.HIGHEST_PROTOCOL)
# The following example reads the resulting pickled data.

In [52]:
with open('Extracted_Features/data.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    final_df = pickle.load(f)

In [53]:
len(final_df["AF3_IMF_1"].iloc[0])

7680

## ANOVA feature extraction

In [31]:
def extract_features(signal):
    signal = cp.array(signal).flatten()
    features = {}
    
    # Basic statistics
    features['Minimum'] = cp.min(signal).get()
    features['Maximum'] = cp.max(signal).get()
    features['Mean'] = cp.mean(signal).get()
    features['MAV'] = cp.mean(cp.abs(signal)).get()
    
    # Enhanced MAV
    p = 0.5  # example parameter for EMAV
    features['EMAV'] = cp.mean(cp.abs(signal)**p).get()
    
    # Modified MAV1 and MAV2
    n = len(signal)
    w1 = cp.hamming(n)  # Example weighting function for MMAV1
    w2 = cp.hanning(n)  # Example weighting function for MMAV2
    features['MMAV1'] = cp.mean(cp.abs(signal) * w1).get()
    features['MMAV2'] = cp.mean(cp.abs(signal) * w2).get()
    
    # Mean Absolute Deviation
    features['MAD'] = cp.mean(cp.abs(signal - cp.mean(signal))).get()
    
    # Root Mean Square
    features['RMS'] = cp.sqrt(cp.mean(signal**2)).get()
    
    # Median and Median Absolute Deviation
    features['Median'] = cp.median(signal).get()
    features['Median_Absolute_Deviation'] = cp.median(cp.abs(signal - cp.median(signal))).get()
    
    # Difference Absolute Standard Deviation Value
    diff_signal = cp.diff(signal)
    features['DASDV'] = cp.sqrt(cp.mean(diff_signal**2)).get()
    
    # Standard Deviation and Variance
    features['STD'] = cp.std(signal).get()
    features['Variance'] = cp.var(signal).get()
    
    # Coefficient Of Variance
    features['COV'] = stats.variation(cp.asnumpy(signal))
    
    # Skewness and Kurtosis
    features['Skewness'] = stats.skew(cp.asnumpy(signal))
    features['Kurtosis'] = stats.kurtosis(cp.asnumpy(signal))
    
    # Waveform Length and Enhanced Waveform Length
    features['Waveform_Length'] = cp.sum(cp.abs(diff_signal)).get()
    features['EWL'] = cp.sum(cp.abs(diff_signal)**p).get()
    
    # Interquartile Range
    features['IQR'] = stats.iqr(cp.asnumpy(signal))
    
    # Average Amplitude Change
    features['AAC'] = cp.mean(cp.abs(diff_signal)).get()
    
    # Log Detector
    features['Log_Detector'] = cp.exp(cp.mean(cp.log(cp.abs(signal)))).get()
    
    # Myopulse percentage rate
    threshold = 0.1  # example threshold for MYOP
    features['MYOP'] = cp.mean(cp.abs(signal) > threshold).get()
    
    # Simple Square Integral
    features['SSI'] = cp.sum(signal**2).get()
    
    # Willison Amplitude
    features['WA'] = cp.sum(cp.abs(diff_signal) > threshold).get()
    
    # Maximum Fractal Length
    features['MFL'] = cp.sum(cp.abs(cp.diff(diff_signal))).get()
    
    # Slope Sign Change
    features['SSC'] = cp.sum(cp.diff(cp.sign(diff_signal)) != 0).get()
    
    # Zero Crossing
    features['ZC'] = cp.sum(cp.diff(cp.sign(signal)) != 0).get()
    
    # Entropy
    features['Entropy'] = stats.entropy(cp.asnumpy(cp.histogram(signal, bins=10, density=True)[0]))
    
    # Shannon Entropy
    bases = collections.Counter([tmp_base for tmp_base in cp.asnumpy(signal)])
    dist = [x / sum(bases.values()) for x in bases.values()]
    entropy_value = stats.entropy(dist, base=2)
    features['Shannon_Entropy'] = entropy_value
    
    # Log Energy Entropy
    features['Log_Energy_Entropy'] = cp.sum(cp.log2(signal**2 + 1e-12)).get()
    
    return features


In [32]:
# Function to extract additional features
# def extract_features(signal):
#     signal = np.array(signal).flatten()
#     if len(signal) == 0:
#         raise ValueError("Signal length is zero. Cannot extract features from an empty signal.")
    
#     # Calculate histogram for entropy
#     hist, _ = np.histogram(signal, bins=10, density=True)
    # bases = collections.Counter([tmp_base for tmp_base in signal])
    # # define distribution
    # dist = [x/sum(bases.values()) for x in bases.values()]
 
    # # use scipy to calculate entropy
    # entropy_value = entropy(dist, base=2)
#     features = {
#         'Variance': np.var(signal),
#         'Mean_absolute_value': np.mean(np.abs(signal)),
#         'Mean_absolute_Deviation': np.mean(np.abs(signal - np.mean(signal))),
#         'Modified_Mean_Absolute_Value': np.mean(np.abs(signal - np.median(signal))),
#         'Modified_Mean_Absolute_Value_squared': np.mean((signal - np.median(signal))**2),
#         'Root_Mean_Square': np.sqrt(np.mean(signal**2)),
#         'Standard_Deviation': np.std(signal),
#         'Inter_Quartile_Range': np.percentile(signal, 75) - np.percentile(signal, 25),
#         'Median_Absolute_Deviation': np.median(np.abs(signal - np.median(signal))),
#         'Enhanced_Mean_Absolute_Value': np.mean(np.abs(signal)) + np.var(signal),
#         'Log_Detector': np.exp(np.mean(np.log(np.abs(signal) + 1))),
#         'Willison_Amplitude': np.sum(np.abs(np.diff(signal)) > 0.5),
#         'Myopulse_Percentage_Rate': np.sum(np.abs(signal) > 0.5 * np.max(np.abs(signal))),
#         'Entropy': entropy(hist),  # adding 1 to avoid log(0)
#         'Kurtosis': kurtosis(signal),
#         'Minimum': np.min(signal),
#         'Maximum': np.max(signal),
#         'Maximum_Fractal_Length': np.max(np.diff(signal)),
#         'Enhanced_Wavelength': np.sum(np.abs(np.diff(signal))) + np.std(signal),
#         'Difference_Absolute_STD_Value': np.std(np.diff(signal)),
#         'Log_Energy_Entropy': np.sum(signal**2 * np.log(signal**2 + 1e-9)),
#         'Average_Amplitude_Change': np.mean(np.abs(np.diff(signal))),
#         'Zero_Crossing': np.sum(np.diff(np.sign(signal)) != 0),
#         'Wavelength': np.sum(np.abs(np.diff(signal))),
#         'Slope_Sign_Change': np.sum(np.diff(np.sign(np.diff(signal))) != 0),
#         'Simple_Square_Integral': np.sum(np.array(signal).flatten()**2),
#         'Skewness': skew(signal),
#         'Mean': np.mean(signal),
#         'Covariance': np.cov(signal).item() if len(signal) > 1 else 0,  # handle case where length is 1
#         'Median': np.median(signal),
#         'Shannon_Entropy': entropy_value,
#     }
#     return features

In [33]:
def feature_extraction_ANOVA_EEG(raw, secs):
    # 128 Hz is the sampling rate for the EEG data
    fs_EEG = 128
    N_EEG = math.ceil(fs_EEG * secs)
    features = []
    # for electrode in eeg_electrodes:
    #     features += [f'{electrode}_theta', f'{electrode}_alpha', f'{electrode}_beta', f'{electrode}_gamma'] + [f'{electrode}_IMF_{i + 1}' for i in range(6)]
    
    additional_features = [
        'Minimum', 'Maximum', 'Mean', 'MAV', 'EMAV',
        'MMAV1', 'MMAV2', 'MAD', 'RMS', 'Median',
        'Median_Absolute_Deviation', 'DASDV', 'STD', 'Variance', 'COV',
        'Skewness', 'Kurtosis', 'Waveform_Length', 'EWL', 'IQR',
        'AAC', 'Log_Detector', 'MYOP', 'SSI', 'WA',
        'MFL', 'SSC', 'ZC', 'Entropy', 'Shannon_Entropy',
        'Log_Energy_Entropy'
    ]

    # Adding additional feature columns for each electrode
    for electrode in eeg_electrodes:
        features += [f'{electrode}_{feat}' for feat in additional_features]
    
    # DataFrame to store features
    columns = ['participant', 'video'] + features + ['Valence', 'Arousal', 'Target_Emotion']
    df = pd.DataFrame(columns=columns)
    
    for participant in range(23):
        for video in range(18):
            feature_values = []
            for i in range(14):
                # basl = raw['DREAMER'][0, 0]['Data'][0, participant]['EEG'][0, 0]['baseline'][0, 0][video, 0][-1 - N_EEG:-1, i]
                stimuli = raw['DREAMER'][0, 0]['Data'][0, participant]['EEG'][0, 0]['stimuli'][0, 0][video, 0][-1 - N_EEG:-1, i]
                # Detrending
                # basl_detrended = detrend_signal(basl)
                # stimuli_detrended = detrend_signal(stimuli)

                # Baseline correction
                # stimuli_corrected = baseline_correction(basl_detrended, stimuli_detrended)

                # Feature extraction
                # DWT_stimuli = dwt_and_band_extraction([], stimuli_corrected)
                # emd_stimuli = emd(stimuli_corrected)

                # Combine features
                # feature_values += DWT_stimuli + emd_stimuli

                # print(stimuli_corrected.shape)
                # print(stimuli_corrected)
                # Additional feature extraction
                additional_features = extract_features([stimuli])
                feature_values += additional_features.values()

            # Assuming Valence and Arousal are stored similarly to the EEG signals
            valence = raw['DREAMER'][0, 0]['Data'][0, participant]['ScoreValence'][0, 0][video, 0]
            arousal = raw['DREAMER'][0, 0]['Data'][0, participant]['ScoreArousal'][0, 0][video, 0]
            target_emotion = ['calmness', 'surprise', 'amusement', 'fear', 'excitement', 'disgust',
                  'happiness', 'anger', 'sadness', 'disgust', 'calmness', 'amusement',
                  'happiness', 'anger', 'fear', 'excitement', 'sadness', 'surprise'][video]

            # Append to feature list
            row = [participant + 1, video + 1] + feature_values + [valence, arousal, target_emotion]
            if len(row) == len(columns):
                df.loc[len(df)] = row
            else:
                print(f'Skipping participant {participant + 1}, video {video + 1}')
                print(len(row))
                print(row)
                print(len(columns))
                print(columns)

    return df

In [34]:
# Assuming `raw` is the dataset loaded and `secs` is the time duration for which features are extracted
secs = 60
features_df = feature_extraction_ANOVA_EEG(raw, secs)

In [35]:
features_df

,participant,video,AF3_Minimum,AF3_Maximum,AF3_Mean,AF3_MAV,AF3_EMAV,AF3_MMAV1,AF3_MMAV2,AF3_MAD,...,AF4_WA,AF4_MFL,AF4_SSC,AF4_ZC,AF4_Entropy,AF4_Shannon_Entropy,AF4_Log_Energy_Entropy,Valence,Arousal,Target_Emotion
0,1,1,4356.410256410257,4404.615384615385,4378.840878739316,4378.840878739316,66.17278010526776,2364.7866992033446,2189.651553156738,7.120057926014937,...,7646,320906.1538461539,3350,0,1.497741,9.359771,184782.44352378973,4,3,calmness
1,1,2,4369.743589743591,4406.153846153847,4388.495659722223,4388.495659722223,66.24569888748094,2369.5342333226126,2193.972370157429,6.364145445212372,...,7632,230905.12820512828,3342,0,1.693107,9.086284,184784.81320490196,3,3,surprise
2,1,3,4337.435897435898,4434.871794871796,4388.313969017095,4388.313969017095,66.244321996509,2369.622894346743,2194.0845400275816,6.73687257250492,...,7625,254767.6923076924,3250,0,0.538431,9.570814,184720.74249589824,5,4,amusement
3,1,4,4374.871794871796,4408.717948717949,4392.684895833333,4392.684895833333,66.27731415849543,2371.8529207592305,2196.1284011875696,5.711627092932068,...,7652,396802.5641025641,3377,0,1.777091,8.957243,184871.98869811487,4,3,fear
4,1,5,4371.282051282052,4408.717948717949,4388.646434294873,4388.646434294873,66.24683626086123,2369.678308289578,2194.1158625499875,6.4444407406016255,...,7666,769175.3846153847,3363,0,1.876939,9.368395,184884.04244843338,4,4,excitement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,4332.307692307693,4443.589743589744,4387.893496260685,4387.893496260685,66.24105736587386,2370.1552176030964,2194.6997151111323,13.511581846927365,...,7349,46690.256410256596,3389,0,1.565753,7.634302,184848.43556516533,2,2,anger
410,23,15,4306.153846153847,4451.794871794872,4386.170539529915,4386.170539529915,66.22799944442265,2367.552219436296,2192.0201916020683,15.211562806045237,...,7360,42058.46153846175,3506,0,1.626279,7.281806,184821.18339847238,2,2,fear
411,23,16,4310.769230769231,4494.871794871796,4384.3759348290605,4384.3759348290605,66.21445466944307,2367.952292543872,2192.611106258203,15.073031468405018,...,7277,37511.28205128222,3539,0,1.546503,7.092030,184759.36438712504,3,3,excitement
412,23,17,4333.846153846154,4516.410256410257,4387.941973824787,4387.941973824787,66.24133361881364,2368.506282015334,2192.9031783797295,15.671569580773713,...,7348,38848.7179487181,3444,0,1.511324,7.420196,184777.3133549014,2,2,sadness


In [36]:
with open('Extracted_Features/Statistical_Features.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(features_df, f, pickle.HIGHEST_PROTOCOL)
# The following example reads the resulting pickled data.

In [37]:
with open('Extracted_Features/Statistical_Features.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df3 = pickle.load(f)

In [39]:
additional_features = [
    'Minimum', 'Maximum', 'Mean', 'MAV', 'EMAV',
    'MMAV1', 'MMAV2', 'MAD', 'RMS', 'Median',
    'Median_Absolute_Deviation', 'DASDV', 'STD', 'Variance', 'COV',
    'Skewness', 'Kurtosis', 'Waveform_Length', 'EWL', 'IQR',
    'AAC', 'Log_Detector', 'MYOP', 'SSI', 'WA',
    'MFL', 'SSC', 'ZC', 'Entropy', 'Shannon_Entropy',
    'Log_Energy_Entropy'
]

features = []
# Adding additional feature columns for each electrode
for electrode in eeg_electrodes:
    features += [f'{electrode}_{feat}' for feat in additional_features]

In [40]:
cnt = 0
for feat in features:
    if features_df[feat].dtype == 'object':
        print(feat)
        cnt+=1
print(cnt)

AF3_Minimum
AF3_Maximum
AF3_Mean
AF3_MAV
AF3_EMAV
AF3_MMAV1
AF3_MMAV2
AF3_MAD
AF3_RMS
AF3_Median
AF3_Median_Absolute_Deviation
AF3_DASDV
AF3_STD
AF3_Variance
AF3_Waveform_Length
AF3_EWL
AF3_AAC
AF3_Log_Detector
AF3_MYOP
AF3_SSI
AF3_WA
AF3_MFL
AF3_SSC
AF3_ZC
AF3_Log_Energy_Entropy
F7_Minimum
F7_Maximum
F7_Mean
F7_MAV
F7_EMAV
F7_MMAV1
F7_MMAV2
F7_MAD
F7_RMS
F7_Median
F7_Median_Absolute_Deviation
F7_DASDV
F7_STD
F7_Variance
F7_Waveform_Length
F7_EWL
F7_AAC
F7_Log_Detector
F7_MYOP
F7_SSI
F7_WA
F7_MFL
F7_SSC
F7_ZC
F7_Log_Energy_Entropy
F3_Minimum
F3_Maximum
F3_Mean
F3_MAV
F3_EMAV
F3_MMAV1
F3_MMAV2
F3_MAD
F3_RMS
F3_Median
F3_Median_Absolute_Deviation
F3_DASDV
F3_STD
F3_Variance
F3_Waveform_Length
F3_EWL
F3_AAC
F3_Log_Detector
F3_MYOP
F3_SSI
F3_WA
F3_MFL
F3_SSC
F3_ZC
F3_Log_Energy_Entropy
FC5_Minimum
FC5_Maximum
FC5_Mean
FC5_MAV
FC5_EMAV
FC5_MMAV1
FC5_MMAV2
FC5_MAD
FC5_RMS
FC5_Median
FC5_Median_Absolute_Deviation
FC5_DASDV
FC5_STD
FC5_Variance
FC5_Waveform_Length
FC5_EWL
FC5_AAC
FC5_Log_Detec

In [41]:
features_df['AF4_Shannon_Entropy']

0      9.359771
1      9.086284
2      9.570814
3      8.957243
4      9.368395
         ...   
409    7.634302
410    7.281806
411    7.092030
412    7.420196
413    7.640084
Name: AF4_Shannon_Entropy, Length: 414, dtype: float64

In [42]:
# Split the DataFrame for arousal and valence separately
features_arousal_df = features_df.drop(columns=['Valence']).copy()
features_valence_df = features_df.drop(columns=['Arousal']).copy()
# Replace Inf and -Inf with NaN
features_arousal_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN values
features_arousal_df.dropna(inplace=True)
# Replace Inf and -Inf with NaN
features_valence_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN values
features_valence_df.dropna(inplace=True)
# Remove constant features
constant_filter = VarianceThreshold(threshold=0)
X_arousal = constant_filter.fit_transform(features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']))
# Update feature names after removing constant features
filtered_feature_names = features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns[constant_filter.get_support()]

In [43]:
filtered_feature_names

Index(['AF3_Minimum', 'AF3_Maximum', 'AF3_Mean', 'AF3_MAV', 'AF3_EMAV',
       'AF3_MMAV1', 'AF3_MMAV2', 'AF3_MAD', 'AF3_RMS', 'AF3_Median',
       ...
       'AF4_MYOP', 'AF4_SSI', 'AF4_WA', 'AF4_MFL', 'AF4_SSC', 'AF4_ZC',
       'AF4_Entropy', 'AF4_Shannon_Entropy', 'AF4_Log_Energy_Entropy',
       'Arousal'],
      dtype='object', length=423)

In [44]:
# Original feature names before removing constant features
original_feature_names = features_arousal_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns

# Mask indicating constant features
constant_mask = np.logical_not(constant_filter.get_support())

# Get the names of constant features
constant_features = original_feature_names[constant_mask]

print("Constant features in X_arousal:")
print(constant_features)
# Convert X_arousal back to DataFrame with updated feature names
X_arousal_df = pd.DataFrame(X_arousal, columns=filtered_feature_names)
# Define the target variable
y_arousal = features_arousal_df['Target_Emotion']

# Perform ANOVA
anova_results_arousal = f_classif(X_arousal_df, y_arousal)
f_values_arousal, p_values_arousal = anova_results_arousal

# Create a DataFrame to hold the results
anova_arousal_df = pd.DataFrame({
    'Feature': filtered_feature_names,
    'F-value': f_values_arousal,
    'p-value': p_values_arousal
})

# Sort the DataFrame by p-value
anova_arousal_df.sort_values(by='p-value', inplace=True)

# Select significant features with p-value < 0.05
significant_features_arousal = anova_arousal_df[anova_arousal_df['p-value'] < 0.05]

Constant features in X_arousal:
Index(['AF3_MYOP', 'AF3_ZC', 'F7_MYOP', 'F7_ZC', 'T7_MYOP', 'T7_ZC', 'P7_MYOP',
       'P7_ZC', 'P8_MYOP', 'P8_ZC', 'F4_MYOP', 'F4_ZC'],
      dtype='object')


In [45]:
anova_arousal_df

,Feature,F-value,p-value
422,Arousal,10.599937,1.516647e-13
361,F8_Maximum,5.674975,7.560027e-07
392,AF4_Maximum,3.869239,2.073579e-04
295,T8_SSC,3.323279,1.075695e-03
171,P7_SSI,2.885535,3.894525e-03
...,...,...,...
11,AF3_DASDV,0.032982,9.999883e-01
47,F7_EWL,0.030000,9.999919e-01
77,F3_IQR,0.023354,9.999970e-01
18,AF3_EWL,0.022928,9.999972e-01


In [46]:
significant_features_arousal

,Feature,F-value,p-value
422,Arousal,10.599937,1.516647e-13
361,F8_Maximum,5.674975,7.560027e-07
392,AF4_Maximum,3.869239,2.073579e-04
295,T8_SSC,3.323279,1.075695e-03
171,P7_SSI,2.885535,3.894525e-03
157,P7_RMS,2.878771,3.971536e-03
293,T8_WA,2.688501,6.859377e-03
400,AF4_Median,2.652288,7.603433e-03
33,F7_EMAV,2.535256,1.057999e-02
372,F8_STD,2.505433,1.150200e-02


In [47]:
constant_filter = VarianceThreshold(threshold=0)
X_valence = constant_filter.fit_transform(features_valence_df.drop(columns=['participant', 'video', 'Target_Emotion']))
# Update feature names after removing constant features
filtered_feature_names = features_valence_df.drop(columns=['participant', 'video', 'Target_Emotion']).columns[constant_filter.get_support()]
# Perform ANOVA for valence
X_valence_df = pd.DataFrame(X_valence, columns=filtered_feature_names)
y_valence = features_valence_df['Target_Emotion']

anova_results_valence = f_classif(X_valence_df, y_valence)
f_values_valence, p_values_valence = anova_results_valence

anova_valence_df = pd.DataFrame({
    'Feature': X_valence_df.columns,
    'F-value': f_values_valence,
    'p-value': p_values_valence
})
anova_valence_df.sort_values(by='p-value', inplace=True)
significant_features_valence = anova_valence_df[anova_valence_df['p-value'] < 0.05]

# The significant features are stored in `significant_features_arousal` and `significant_features_valence`

In [48]:
anova_valence_df

,Feature,F-value,p-value
422,Valence,65.173755,4.404425e-68
361,F8_Maximum,5.674975,7.560027e-07
392,AF4_Maximum,3.869239,2.073579e-04
295,T8_SSC,3.323279,1.075695e-03
171,P7_SSI,2.885535,3.894525e-03
...,...,...,...
11,AF3_DASDV,0.032982,9.999883e-01
47,F7_EWL,0.030000,9.999919e-01
77,F3_IQR,0.023354,9.999970e-01
18,AF3_EWL,0.022928,9.999972e-01


In [49]:
significant_features_valence

,Feature,F-value,p-value
422,Valence,65.173755,4.404425e-68
361,F8_Maximum,5.674975,7.560027e-07
392,AF4_Maximum,3.869239,2.073579e-04
295,T8_SSC,3.323279,1.075695e-03
171,P7_SSI,2.885535,3.894525e-03
157,P7_RMS,2.878771,3.971536e-03
293,T8_WA,2.688501,6.859377e-03
400,AF4_Median,2.652288,7.603433e-03
33,F7_EMAV,2.535256,1.057999e-02
372,F8_STD,2.505433,1.150200e-02


In [54]:
valence_columns_to_add = list(significant_features_valence['Feature'])
# Check if final_df already has these columns
existing_columns_valence = final_df.columns.intersection(valence_columns_to_add)
new_columns_valence = [col for col in valence_columns_to_add if col not in existing_columns_valence]
final_valence_df = pd.concat([final_df, features_valence_df[new_columns_valence]], axis=1)

In [55]:
final_valence_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_STD,O2_Maximum,F8_Skewness,P7_MMAV1,T8_SSI,T8_RMS,F7_Skewness,T7_Entropy,P7_MMAV2,F7_MMAV1
0,1,1,"[-0.0390636464244261, 0.09988633859337229, -0....","[0.15276303577969383, 1.2696051548240934, 1.04...","[1.856929710807957, 10.791781431213384, 14.034...","[3.2594967503673904, -1.796087448369993, -0.32...","[-20.945664729800182, -3.585145972089575, 4.35...","[20.079570340536844, 12.845106670083682, 7.499...","[-472.3937889410268, -441.7314862942789, -412....","[807.2982350705635, 760.702080493103, 715.9087...",...,93.87532155055462,4396.923076923077,0.876091,2369.536754533684,140252034062.8534,4273.404217007096,0.240758,1.498082,2194.6654905745168,2211.589264832751
1,1,2,"[0.4447054345237156, -2.2739034949040344, 2.20...","[-1.29806126965089, -10.057003013278859, -7.98...","[0.049750625457719196, 16.734604267689296, 5.2...","[2.059528878325656, -2.1951011266790372, 2.666...","[3.2907522198672656, 9.558369126486213, 5.4292...","[-68.50388036946528, -60.83343124236948, -53.8...","[287.55871187843275, 265.7149737780089, 244.93...","[-306.1977954610037, -285.01993005917274, -264...",...,72.73404672608083,4200.512820512821,0.130204,2349.9227124374784,140174701208.94153,4272.225909278939,-0.034241,1.685100,2175.7115022778235,2216.101350362217
2,1,3,"[0.1450676936889263, 0.3323767388122518, 0.725...","[-0.10225776435297178, -1.8531720250660215, -0...","[-1.5964275091820885, -9.08784206257223, -14.0...","[-3.2885028522031257, 0.39521514219349035, 3.5...","[4.572573342644084, -2.958668674311242, -8.078...","[-58.45535598399263, -49.54152737215601, -41.5...","[179.6804531547695, 158.94030718637714, 139.92...","[-158.39189250236828, -145.03333315846984, -13...",...,205.7900107733026,6067.179487179488,0.025053,2339.267205281654,139234260715.0559,4257.870480722287,0.241397,0.549103,2166.2174715993447,2217.9864467298626
3,1,4,"[0.02241420804875787, 1.1307988471909174, 0.12...","[0.12598080801138814, -0.20322530118359053, 0....","[2.091279813251027, 13.599228413503267, 15.936...","[3.9362192425632094, -1.4331300435338477, -0.6...","[-14.13033785077942, 1.6262765249300335, 8.636...","[317.9041468923732, 278.43980546262344, 242.82...","[-171.79459950088759, -148.3269975635026, -126...","[-171.59902290367822, -156.8806113223296, -143...",...,65.65116742894955,4139.48717948718,0.035344,2351.064674593931,139749457604.73376,4265.740728050605,-0.137400,1.843706,2176.9992804950757,2221.992926716834
4,1,5,"[1.0709170021849068, 4.750019879969691, 6.0586...","[-0.6217449925448907, -14.325770276644187, -3....","[1.069408056436174, 21.515804657065107, 8.8033...","[3.143577123732196, -2.62735610202805, 2.87790...","[5.444857516311669, 13.90708797820107, 9.48599...","[80.0422016774792, 69.74254636781487, 59.82124...","[-116.43984695695993, -99.4370175539302, -84.0...","[666.7787873551133, 635.2689333389025, 604.836...",...,88.50488257251762,4206.666666666667,-0.084295,2346.8782588096956,139604249329.3886,4263.523968670846,0.011300,1.889704,2173.0186213836337,2220.4998849011067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[-1.6257516966485197, -22.549341769905883, -10...","[-1.8307911457716481, 14.076797310493374, -11....","[-1.7194771272843405, -6.5520805882942765, 0.0...","[-0.7243271988767334, 1.0256966662912683, 0.33...","[-89.82175150696582, -65.30009587833204, -41.5...","[258.17014084945714, 204.57960244020467, 159.7...","[-179.44886447907598, -151.25541164395247, -12...","[-148.7429926265613, -143.49484989029142, -138...",...,30.719710724250405,3988.205128205129,-0.059190,2359.077234131827,140646881235.766,4279.415379960254,0.198845,1.842790,2184.293015081974,2218.5073685598686
410,23,15,"[-0.34769011135992756, -9.896798418848553, -2....","[-1.976691111166075, -4.525220086587955, -12.1...","[-2.1216863281309988, 3.314999480896199, -6.44...","[-1.124054294663887, -0.6437197702755763, 0.64...","[-1

In [56]:
with open('Extracted_Features/Final_Valence_Features.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(final_valence_df, f, pickle.HIGHEST_PROTOCOL)
# The following example reads the resulting pickled data.

In [57]:
with open('Extracted_Features/Final_Valence_Features.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df4 = pickle.load(f)

In [58]:
arousal_columns_to_add = list(significant_features_arousal['Feature'])
existing_columns_arousal = final_df.columns.intersection(arousal_columns_to_add)
new_columns_arousal = [col for col in arousal_columns_to_add if col not in existing_columns_arousal]
final_arousal_df = pd.concat([final_df, features_arousal_df[new_columns_arousal]], axis=1)

In [59]:
final_arousal_df

,participant,video,AF3_theta,AF3_alpha,AF3_beta,AF3_gamma,AF3_IMF_1,AF3_IMF_2,AF3_IMF_3,AF3_IMF_4,...,AF4_STD,O2_Maximum,F8_Skewness,P7_MMAV1,T8_SSI,T8_RMS,F7_Skewness,T7_Entropy,P7_MMAV2,F7_MMAV1
0,1,1,"[-0.0390636464244261, 0.09988633859337229, -0....","[0.15276303577969383, 1.2696051548240934, 1.04...","[1.856929710807957, 10.791781431213384, 14.034...","[3.2594967503673904, -1.796087448369993, -0.32...","[-20.945664729800182, -3.585145972089575, 4.35...","[20.079570340536844, 12.845106670083682, 7.499...","[-472.3937889410268, -441.7314862942789, -412....","[807.2982350705635, 760.702080493103, 715.9087...",...,93.87532155055462,4396.923076923077,0.876091,2369.536754533684,140252034062.8534,4273.404217007096,0.240758,1.498082,2194.6654905745168,2211.589264832751
1,1,2,"[0.4447054345237156, -2.2739034949040344, 2.20...","[-1.29806126965089, -10.057003013278859, -7.98...","[0.049750625457719196, 16.734604267689296, 5.2...","[2.059528878325656, -2.1951011266790372, 2.666...","[3.2907522198672656, 9.558369126486213, 5.4292...","[-68.50388036946528, -60.83343124236948, -53.8...","[287.55871187843275, 265.7149737780089, 244.93...","[-306.1977954610037, -285.01993005917274, -264...",...,72.73404672608083,4200.512820512821,0.130204,2349.9227124374784,140174701208.94153,4272.225909278939,-0.034241,1.685100,2175.7115022778235,2216.101350362217
2,1,3,"[0.1450676936889263, 0.3323767388122518, 0.725...","[-0.10225776435297178, -1.8531720250660215, -0...","[-1.5964275091820885, -9.08784206257223, -14.0...","[-3.2885028522031257, 0.39521514219349035, 3.5...","[4.572573342644084, -2.958668674311242, -8.078...","[-58.45535598399263, -49.54152737215601, -41.5...","[179.6804531547695, 158.94030718637714, 139.92...","[-158.39189250236828, -145.03333315846984, -13...",...,205.7900107733026,6067.179487179488,0.025053,2339.267205281654,139234260715.0559,4257.870480722287,0.241397,0.549103,2166.2174715993447,2217.9864467298626
3,1,4,"[0.02241420804875787, 1.1307988471909174, 0.12...","[0.12598080801138814, -0.20322530118359053, 0....","[2.091279813251027, 13.599228413503267, 15.936...","[3.9362192425632094, -1.4331300435338477, -0.6...","[-14.13033785077942, 1.6262765249300335, 8.636...","[317.9041468923732, 278.43980546262344, 242.82...","[-171.79459950088759, -148.3269975635026, -126...","[-171.59902290367822, -156.8806113223296, -143...",...,65.65116742894955,4139.48717948718,0.035344,2351.064674593931,139749457604.73376,4265.740728050605,-0.137400,1.843706,2176.9992804950757,2221.992926716834
4,1,5,"[1.0709170021849068, 4.750019879969691, 6.0586...","[-0.6217449925448907, -14.325770276644187, -3....","[1.069408056436174, 21.515804657065107, 8.8033...","[3.143577123732196, -2.62735610202805, 2.87790...","[5.444857516311669, 13.90708797820107, 9.48599...","[80.0422016774792, 69.74254636781487, 59.82124...","[-116.43984695695993, -99.4370175539302, -84.0...","[666.7787873551133, 635.2689333389025, 604.836...",...,88.50488257251762,4206.666666666667,-0.084295,2346.8782588096956,139604249329.3886,4263.523968670846,0.011300,1.889704,2173.0186213836337,2220.4998849011067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,23,14,"[-1.6257516966485197, -22.549341769905883, -10...","[-1.8307911457716481, 14.076797310493374, -11....","[-1.7194771272843405, -6.5520805882942765, 0.0...","[-0.7243271988767334, 1.0256966662912683, 0.33...","[-89.82175150696582, -65.30009587833204, -41.5...","[258.17014084945714, 204.57960244020467, 159.7...","[-179.44886447907598, -151.25541164395247, -12...","[-148.7429926265613, -143.49484989029142, -138...",...,30.719710724250405,3988.205128205129,-0.059190,2359.077234131827,140646881235.766,4279.415379960254,0.198845,1.842790,2184.293015081974,2218.5073685598686
410,23,15,"[-0.34769011135992756, -9.896798418848553, -2....","[-1.976691111166075, -4.525220086587955, -12.1...","[-2.1216863281309988, 3.314999480896199, -6.44...","[-1.124054294663887, -0.6437197702755763, 0.64...","[-1

In [60]:
with open('Extracted_Features/Final_Arousal_Features.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(final_arousal_df, f, pickle.HIGHEST_PROTOCOL)
# The following example reads the resulting pickled data.

In [61]:
with open('Extracted_Features/Final_Arousal_Features.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df4 = pickle.load(f)